fix

In [ ]:
import yfinance as yf
import pandas as pd
import warnings
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt

In [ ]:
# Suppress FutureWarning
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# Function to get stock data
def get_stock_data(ticker):
    stock = yf.Ticker(ticker)
    data = stock.history(period="max")
    data.reset_index(inplace=True)
    data['Date'] = pd.to_datetime(data['Date']).dt.tz_localize(None)
    return data

In [ ]:
# Function to get revenue data
def get_revenue_data(url, table_index, date_format=None):
    response = requests.get(url)
    html_data = response.text
    soup = BeautifulSoup(html_data, 'html.parser')
    table = soup.find_all('table')[table_index]
    revenue = pd.read_html(str(table))[0]
    revenue.columns = ['Date', 'Revenue']
    # Print the raw revenue data for debugging
    print("Raw Revenue Data:")
    print(revenue.head())
    # Ensure Date column is in datetime format
    if date_format:
        revenue['Date'] = pd.to_datetime(revenue['Date'], format=date_format, errors='coerce').dt.tz_localize(None)
    else:
        revenue['Date'] = pd.to_datetime(revenue['Date'], errors='coerce').dt.tz_localize(None)
    # Clean the Revenue column
    revenue['Revenue'] = revenue['Revenue'].str.replace(',', '').str.replace('$', '').astype(float)
    # Drop null or empty values
    revenue.dropna(inplace=True)
    return revenue

In [ ]:
# Function to create a graph
def make_graph(stock_data, revenue_data, title):
    fig, ax1 = plt.subplots(figsize=(14, 8))
    
    ax1.plot(stock_data['Date'], stock_data['Close'], 'b-')
    ax1.set_xlabel('Date')
    ax1.set_ylabel('Stock Price', color='b')
    ax1.tick_params('y', colors='b')

    ax2 = ax1.twinx()
    ax2.plot(revenue_data['Date'], revenue_data['Revenue'], 'r-')
    ax2.set_ylabel('Revenue', color='r')
    ax2.tick_params('y', colors='r')

    plt.title(title)
    plt.show()

In [ ]:
# Fetch and prepare data for Tesla and GameStop
tesla_data = get_stock_data("TSLA")
gme_data = get_stock_data("GME")
tesla_revenue = get_revenue_data("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm", 0, date_format='%Y')
gme_revenue = get_revenue_data("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html", 1)

In [ ]:
# Display the first five rows
print("Tesla Data:")
print(tesla_data.head())

In [ ]:
# Display the first five rows
print("\nGameStop Data:")
print(gme_data.head())

In [ ]:
# Filter Tesla data and revenue starting from 2010
tesla_data = tesla_data[tesla_data['Date'] >= pd.Timestamp('2010-01-01')]
tesla_revenue = tesla_revenue[tesla_revenue['Date'] >= pd.Timestamp('2010-01-01')]

In [ ]:
# Display the last 5 rows of the tesla_revenue DataFrame
print("Tesla Revenue:")
print(tesla_revenue.tail())

In [ ]:
# Display the last 5 rows of the gme_revenue DataFrame
print("\nGameStop Revenue:")
print(gme_revenue.tail())

In [ ]:
# Generate graphs for Tesla and GameStop
make_graph(tesla_data[tesla_data['Date'] <= pd.Timestamp('2021-06-30')], tesla_revenue[tesla_revenue['Date'] <= pd.Timestamp('2021-06-30')], 'Tesla')
make_graph(gme_data[gme_data['Date'] <= pd.Timestamp('2021-06-30')], gme_revenue[gme_revenue['Date'] <= pd.Timestamp('2021-06-30')], 'GameStop')